# Импорт библиотек

In [54]:
import numpy as np
import pandas as pd
import re

import os

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

import matplotlib
from matplotlib import pyplot as plt

from catboost import CatBoostRegressor

import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras import layers as L
from keras.callbacks import ModelCheckpoint
from keras.callbacks import EarlyStopping
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras import Model

import spacy
from spacy.lang.ru.examples import sentences
from spacy.lang.ru import Russian

import nltk
nltk.download("stopwords")

from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

import PIL
from PIL import Image, ImageDraw 
import matplotlib as mpl
import matplotlib.pyplot as plt
import cv2

!pip install -U albumentations
import albumentations
from albumentations import (
    HorizontalFlip, IAAPerspective, ShiftScaleRotate, CLAHE, RandomRotate90,
    Transpose, ShiftScaleRotate, Blur, OpticalDistortion, GridDistortion, HueSaturationValue,
    IAAAdditiveGaussianNoise, GaussNoise, MotionBlur, MedianBlur, IAAPiecewiseAffine,
    IAASharpen, IAAEmboss, RandomBrightnessContrast, Flip, OneOf, Compose
)

from tensorflow import data as DATA
from tensorflow.data import Dataset

In [55]:
!python -m spacy download ru_core_news_md
spacy.load("ru_core_news_md")

# Параметры

In [56]:
DATA_DIR = '../input/sf-dst-car-price-prediction-part2/'
random_seed = 42

# TOKENIZER
# the maximum number of words to be used. (most frequent)
MAX_WORDS = 100000
# Max number of words in each complaint
MAX_SEQUENCE_LENGTH = 256

size = (320, 240)

# Функции

In [57]:
# Расчет mape
def mape(Y_actual,Y_Predicted):
    mape = np.mean(np.abs((Y_actual - Y_Predicted)/Y_actual))
    return mape

# Визуализация данных в виде гистограммы
def visualize_distributions(titles_values_dict):
  columns = min(3, len(titles_values_dict))
  rows = (len(titles_values_dict) - 1) // columns + 1
  fig = plt.figure(figsize = (columns * 6, rows * 4))
  for i, (title, values) in enumerate(titles_values_dict.items()):
    hist, bins = np.histogram(values, bins = 20)
    ax = fig.add_subplot(rows, columns, i + 1)
    ax.bar(bins[:-1], hist, width = (bins[1] - bins[0]) * 0.7)
    ax.set_title(title)
  plt.show()

# Предобработка данных
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # 1. Предобработка 
    # убираем не нужные для модели признаки
    df_output.drop(['description','sell_id',], axis = 1, inplace=True)
    
    
    # Numerical Features  
    # Далее заполняем пропуски
    for column in numerical_features:
        df_output[column].fillna(df_output[column].median(), inplace=True)
    
    # Нормализация данных
    scaler = MinMaxScaler()
    for column in numerical_features:
        df_output[column] = scaler.fit_transform(df_output[[column]])[:,0]
    
    
    
    # Categorical Features  
    # Label Encoding
    for column in categorical_features:
        df_output[column] = df_output[column].astype('category').cat.codes
        
    # One-Hot Encoding: в pandas есть готовая функция - get_dummies.
    df_output = pd.get_dummies(df_output, columns=categorical_features, dummy_na=False)
    # убираем признаки которые еще не успели обработать, 
    #df_output.drop(['vehicleConfiguration'], axis = 1, inplace=True)
    
    return df_output


def object_to_float (df, column):
    try:
        for i in range(len(df)):
            if df[column][i] == 'undefined LTR':
                df[column][i] = np.float(2.0)
            elif df[column][i] == 'nan' or df[column][i] == ''or df[column][i] == ' ':
                df[column][i] = df[column][i-1]
            elif type(df[column][i]) == float:
                df[column][i] = df[column][i-1]
            else:
                df[column][i] = np.float(re.findall('[-+]?(?:\d+(?:\.\d*)?|\.\d+)(?:[eE][-+]?\d+)?', 
                                                             df[column][i])[0])
    except TypeError:
        print('проблема в', i)
    print(df[column].describe())

def clean_lemmatization (df):
    bad_words = ['й', 'птс', 'км', 'дтп', 'два', 'рольф', 'г', 'со', 'ни', 'под', 'лс', 'м', 'р', 'л', 'тыс', 'рф', 'х', 'вс', 'тд', 
                 'кпп', 'тп', 'итд', 'е', 'ти', 'т', 'ещ', 'гк', 'оот', 'двс', 'три', 'пд', 'ог', 'ткм', 'см', 'тк', 'лкп', 'грм',
                'бу','оф','рус']
    nlp = spacy.load('ru_core_news_md')
    reg = re.compile('[^а-яА-Я ]')
    for i in range(len(df)):
        string = df['description'][i]
        string = re.sub('\n{1,2}', ' ', string)
        string = reg.sub('', string)
        doc = nlp(string)
        clean_and_lemm_string = []
        for token in doc:
            if token.text not in bad_words and token.is_stop == False and token.pos_ != 'DET' and token.pos_ != 'SPACE':
                clean_and_lemm_string.append(token.lemma_)
        clean_and_lemm_string = " ".join(clean_and_lemm_string)
        clean_and_lemm_string = clean_and_lemm_string.strip()
        df['description'][i] = clean_and_lemm_string
        if i%500 == 0:
            print ('Обработано', i, 'строк из', len(df))
            
def get_image_array(index):
    images_train = []
    for index, sell_id in enumerate(data['sell_id'].iloc[index].values):
        image = cv2.imread(DATA_DIR + 'img/img/' + str(sell_id) + '.jpg')
        assert(image is not None)
        image = cv2.resize(image, size)
        images_train.append(image)
    images_train = np.array(images_train)
    print('images shape', images_train.shape, 'dtype', images_train.dtype)
    return(images_train)

# Считываем данные

In [58]:
test = pd.read_csv('../input/sf-dst-car-price-prediction-part2/test.csv')
train = pd.read_csv('../input/sf-dst-car-price-prediction-part2/train.csv')
sample_submission = pd.read_csv('../input/sf-dst-car-price-prediction-part2/sample_submission.csv')

# Предобработка данных

## Объединим датасеты

In [59]:
train['sample'] = 1 # помечаем где у нас трейн
test['sample'] = 0 # помечаем где у нас тест
test['price'] = 0 # в тесте у нас нет значения price, мы его должны предсказать, по этому пока просто заполняем нулями

data = test.append(train, sort=False).reset_index(drop=True) # объединяем
print(train.shape, test.shape, data.shape)

## Проверим наличие дубликатов

In [60]:
len(data.drop_duplicates()) == len(data)
# дубликатов нет

## Посмотрим на данные

In [61]:
data.info()

Не все поля колонки владение заполнены. Посмотрим.

Заполнено 2935 из 8353 значений. Можно заняться восстановлением информации, но кажется усилия будут не оправданы. Удалим колонку.

In [62]:
data = data.drop('Владение', axis = 1)
data.columns

Переведем некоторые колонки в числовые

In [63]:
object_to_float(data, 'engineDisplacement')

In [64]:
object_to_float(data, 'enginePower')

Колонка vehicleConfiguration содержит дублирующую информацию, удалим ее

In [65]:
data['vehicleConfiguration'].value_counts()

In [66]:
data = data.drop('vehicleConfiguration', axis = 1)
data.columns

In [67]:
data['Владельцы'].value_counts()

In [68]:
object_to_float(data, 'Владельцы')

In [69]:
data = data.drop('name', axis = 1)
data.columns

# Предобработка данных

## Добавим новые признаки

In [70]:
data['model_prod_date'] = data['modelDate'] - data['productionDate']
data['engineDisplacement_enginePower'] = data['engineDisplacement'] / data['enginePower']

## Посмотрим на распределение данных

In [71]:
#посмотрим, как выглядят распределения числовых признаков

visualize_distributions({
    'mileage': data['mileage'].dropna(),
    'modelDate': data['modelDate'].dropna(),
    'productionDate': data['productionDate'].dropna(),
    'enginePower' : data['enginePower'].dropna(),
    'engineDisplacement' : data['engineDisplacement'].dropna(),
    'Владельцы' : data['Владельцы'].dropna(),
    'model_prod_date' : data['model_prod_date'].dropna(),
    'engineDisplacement_enginePower': data['engineDisplacement_enginePower'].dropna()
})

In [72]:
#используем все текстовые признаки как категориальные без предобработки
categorical_features = ['bodyType', 'brand', 'color', 'fuelType', 'model_info',
  'numberOfDoors', 'vehicleTransmission', 'ПТС', 'Привод', 'Руль']

#используем все числовые признаки
numerical_features = ['mileage', 'modelDate', 'productionDate', 'engineDisplacement', 'enginePower', 'Владельцы',
                     'engineDisplacement_enginePower', 'model_prod_date']

## Предобработаем данные

In [73]:
data['modelDate'] = np.log(2021 - data['modelDate'])
data['modelDate'].hist(bins = 50)

In [74]:
data['productionDate'] = np.log(2021 - data['productionDate'])
data['productionDate'].hist(bins = 50)

In [75]:
data['mileage'] = np.log(data['mileage'])
data['mileage'].hist(bins = 50)

In [76]:
#посмотрим, как выглядят распределения числовых признаков

visualize_distributions({
    'mileage': data['mileage'].dropna(),
    'modelDate': data['modelDate'].dropna(),
    'productionDate': data['productionDate'].dropna(),
    'enginePower' : data['enginePower'].dropna(),
    'engineDisplacement' : data['engineDisplacement'].dropna(),
    'Владельцы' : data['Владельцы'].dropna(),
    'model_prod_date' : data['model_prod_date'].dropna(),
    'engineDisplacement_enginePower': data['engineDisplacement_enginePower'].dropna()
})

In [77]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [78]:
#посмотрим, как выглядят распределения числовых признаков

visualize_distributions({
    'mileage': df_preproc['mileage'].dropna(),
    'modelDate': df_preproc['modelDate'].dropna(),
    'productionDate': df_preproc['productionDate'].dropna(),
    'enginePower' : df_preproc['enginePower'].dropna(),
    'engineDisplacement' : df_preproc['engineDisplacement'].dropna(),
    'Владельцы' : df_preproc['Владельцы'].dropna(),
    'model_prod_date' : df_preproc['model_prod_date'].dropna(),
    'engineDisplacement_enginePower': df_preproc['engineDisplacement_enginePower'].dropna()
})

## Сформируем датафреймы для обучения

In [79]:
train_data = df_preproc.query('sample == 1').drop(['sample'], axis=1)
test_data = df_preproc.query('sample == 0').drop(['sample'], axis=1)

Прологарифмируем цену

In [80]:
train_data['price'] = np.log(train_data['price'])
train_data['price'].hist(bins = 100)

In [81]:
y = train_data.price.values     # наш таргет
X = train_data.drop(['price'], axis=1)
X_sub = test_data.drop(['price'], axis=1)

### Разделим на тестовую и тренировочную выборки

In [82]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, shuffle=True, random_state=random_seed)

# Градиентный бустинг

In [83]:
model_cb = CatBoostRegressor(iterations = 5000,
                          #depth=10,
                          #learning_rate = 0.5,
                          random_seed = random_seed,
                          eval_metric='MAPE',
                          custom_metric=['RMSE', 'MAE'],
                          od_wait=500,
                          #task_type='GPU',
                         )
model_cb.fit(X_train, y_train,
         eval_set=(X_test, y_test),
         verbose_eval=500,
         use_best_model=True,
         #plot=True
         )

In [84]:
test_predict_catboost = model_cb.predict(X_test)
test_predict_catboost = np.exp(test_predict_catboost)
test_predict_catboost = test_predict_catboost.round(-3)
print(f"TEST mape: {(mape(np.exp(y_test), test_predict_catboost))*100:0.2f}%")

In [85]:
sub_predict_catboost = model_cb.predict(X_sub)
sub_predict_catboost = np.exp(sub_predict_catboost)
sub_predict_catboost = sub_predict_catboost.round(-3)
sample_submission['price'] = sub_predict_catboost
sample_submission.to_csv('catboost_submission3.csv', index=False)

# Обработка текста

## Соберем текстовые данные в одну серию

In [86]:
text_series = data[['description']]
text_series.sample(5)

## Почитстим текст. Лемматизируем его.

In [ ]:
#%%time 
#clean_lemmatization(text_series)

In [ ]:
text_series.to_csv('lemmatizated_description_full.csv')

In [87]:
text_series = pd.read_csv('../input/lematizated-car-description/lemmatizated_description_full-2.csv', index_col = 0)

In [88]:
for i in range (len(text_series)):
    if type(text_series['description'][i]) == float:
        text_series['description'][i] = 'нет описание'
        print(i)

## Токенезируем текст

In [89]:
%%time 
tokenize = Tokenizer(num_words = MAX_WORDS)
tokenize.fit_on_texts(text_series['description'])

# Обработка картинок

In [90]:
plt.figure(figsize = (12,8))

random_image = train.sample(n = 9)
random_image_paths = random_image['sell_id'].values
random_image_cat = random_image['price'].values

for index, path in enumerate(random_image_paths):
    im = PIL.Image.open(DATA_DIR+'img/img/' + str(path) + '.jpg')
    plt.subplot(3, 3, index + 1)
    plt.imshow(im)
    plt.title('price: ' + str(random_image_cat[index]))
    plt.axis('off')
plt.show()

In [91]:
images_train = get_image_array(X_train.index)
images_test = get_image_array(X_test.index)
images_sub = get_image_array(X_sub.index)

In [92]:
augmentation = Compose([
    HorizontalFlip(),
    GaussNoise(var_limit=(1000.0, 5000.0), p=0.3),
    
    OneOf([
        MotionBlur(blur_limit = (5,7)),
        MedianBlur(blur_limit=5),
        Blur(blur_limit=5)
    ], p=0.3),
    
    ShiftScaleRotate(shift_limit=0.05, scale_limit=0.2, rotate_limit=5),
    
    OneOf([
        OpticalDistortion(distort_limit=0.2, shift_limit=0.2, interpolation=1, border_mode=4),
        GridDistortion(num_steps=1, distort_limit=0.1)
    ], p = 0.3),
    
    OneOf([
        CLAHE(clip_limit=4.0, tile_grid_size=(8, 8)),
        IAASharpen(),
        IAAEmboss(),
        RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2)
    ], p = 0.3),
    HueSaturationValue(hue_shift_limit=30, sat_shift_limit=30, val_shift_limit=30, p = 0.3),
], p=0.7)

#пример
plt.figure(figsize = (12,8))
for i in range(9):
    img = augmentation(image = images_train[10])['image']
    plt.subplot(3, 3, i + 1)
    plt.imshow(img)
    plt.axis('off')
plt.show()

In [93]:
def make_augmentations(images):
  print('применение аугментаций', end = '')
  augmented_images = np.empty(images.shape)
  for i in range(images.shape[0]):
    if i % 200 == 0:
      print('.', end = '')
    augment_dict = augmentation(image = images[i])
    augmented_image = augment_dict['image']
    augmented_images[i] = augmented_image
  print('')
  return augmented_images

Потенциируем цену. Нейронке плохо с логарифимрованной ценой.

In [94]:
y_train = np.exp(y_train)
y_train

In [95]:
y_test = np.exp(y_test)
y_test

In [96]:
def process_image(image):
    return augmentation(image = image.numpy())['image']

def tokenize_(descriptions):
  return sequence.pad_sequences(tokenize.texts_to_sequences(descriptions), maxlen = MAX_SEQUENCE_LENGTH)

def tokenize_text(text):
    return tokenize_([text.numpy().decode('utf-8')])[0]

def tf_process_train_dataset_element(image, table_data, text, price):
    im_shape = image.shape
    [image,] = tf.py_function(process_image, [image], [tf.uint8])
    image.set_shape(im_shape)
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

def tf_process_val_dataset_element(image, table_data, text, price):
    [text,] = tf.py_function(tokenize_text, [text], [tf.int32])
    return (image, table_data, text), price

train_dataset = tf.data.Dataset.from_tensor_slices((
    images_train, X_train, text_series.description.iloc[X_train.index], y_train
    )).map(tf_process_train_dataset_element)

print('train_dataset done')

test_dataset = tf.data.Dataset.from_tensor_slices((
    images_test, X_test, text_series.description.iloc[X_test.index], y_test
    )).map(tf_process_val_dataset_element)

print('test_dataset done')

y_sub = np.zeros(len(X_sub))
sub_dataset = tf.data.Dataset.from_tensor_slices((
    images_sub, X_sub, text_series.description.iloc[X_sub.index], y_sub
    )).map(tf_process_val_dataset_element)

print('sub_dataset done')

#проверяем, что нет ошибок (не будет выброшено исключение):
train_dataset.__iter__().__next__();
print('train_dataset done')
test_dataset.__iter__().__next__();
print('test_dataset done')
sub_dataset.__iter__().__next__();
print('sub_dataset done')

Далее я взял архитектуру baseline. Я пытался ее улучшить, но не смог побороть переобучение.
Я даже не знаю скольк времени надо, чтобы попробовать различные архитектуры, но у меня сейчас столько точно нет...

## Зададим архитектуру сети для работы с картинками

In [97]:
efficientnet_model = tf.keras.applications.efficientnet.EfficientNetB3(weights = 'imagenet', include_top = False, input_shape = (size[1], size[0], 3))
efficientnet_output = L.GlobalAveragePooling2D()(efficientnet_model.output)

##  Зададим архитектуру сети для работы с табличными данными

In [98]:
tabular_model = Sequential([
    L.Input(shape = X.shape[1]),
    L.Dense(512, activation = 'relu'),
    L.Dropout(0.25),
    L.Dense(256, activation = 'relu'),
    L.Dropout(0.25),
    ])

## Зададим архитектуру сети для работы с текстом

In [99]:
nlp_model = Sequential([
    L.Input(shape=MAX_SEQUENCE_LENGTH, name="seq_description"),
    L.Embedding(len(tokenize.word_index)+1, MAX_SEQUENCE_LENGTH,),
    L.LSTM(256, return_sequences=True),
    L.Dropout(0.25),
    L.LSTM(128),
    L.Dropout(0.25),
    L.Dense(64),
    ])

## Скомбинируем сети

In [100]:
combinedInput = L.concatenate([efficientnet_output, tabular_model.output, nlp_model.output])

# being our regression head
#head = L.Dense(512, activation="relu")
#head = L.Dropout(0.25)
head = L.Dense(256, activation="relu")(combinedInput)
head = L.Dense(1,)(head)

model = Model(inputs=[efficientnet_model.input, tabular_model.input, nlp_model.input], outputs=head)
#model.summary()

## Зададим настройки обучения

Я отключил расписание ЛР, чтобы решение быстрее сходилось. Однак, скорее всего, это плохо, и в перспективе наличие расписания дало бы плюс...

In [101]:
optimizer = tf.keras.optimizers.Adam(0.005)
#lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#    initial_learning_rate=0.005,
#    decay_steps=190,
#    decay_rate=0.1)
#optimizer = keras.optimizers.Adam(learning_rate=lr_schedule)

model.compile(loss='MAPE',optimizer=optimizer, metrics=['MAPE'])

In [102]:
checkpoint = ModelCheckpoint('../working/best_model_1.hdf5', monitor=['val_MAPE'], verbose=0, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_MAPE', factor=0.5,patience=3, min_lr=0.00000001, min_delta = 0.2)
earlystop = EarlyStopping(monitor='val_MAPE', patience=5, restore_best_weights=True, min_delta = 0.1)
callbacks_list = [checkpoint, earlystop, reduce_lr]

## Обучим модель

Я не буду делать файнтюнинг из-за недостатка времени. Файнтюнинг освоил на предыдущем проекте.

In [103]:
history = model.fit(train_dataset.batch(30),
                    epochs = 100,
                    validation_data = test_dataset.batch(30),
                    callbacks = callbacks_list
)

In [104]:
model.save('../working/nn_final_2_output.hdf5')
#model.load_weights('../working/best_model_1.hdf5')
#model.save('../working/nn_final_2_input.hdf5')

In [105]:
test_predict_nn3 = model.predict(test_dataset.batch(30))
test_predict_nn3 = test_predict_nn3.round(-3)
print(f"TEST mape: {(mape(y_test, test_predict_nn3[:,0]))*100:0.2f}%")

In [111]:
sub_predict_nn3 = model.predict(sub_dataset.batch(30))
sub_predict_nn3 = sub_predict_nn3.round(-3)
#sample_submission['price'] = sub_predict_nn3
#sample_submission.to_csv('nn3_submission3.csv', index=False)

In [110]:
blend_predict = (test_predict_catboost + test_predict_nn3[:,0]) / 2
blend_predict = blend_predict.round(-3)
print(f"TEST mape: {(mape(y_test, blend_predict))*100:0.2f}%")

In [114]:
blend_predict_sub = (sub_predict_catboost + sub_predict_nn3[:,0]) / 2
blend_predict_sub = blend_predict_sub.round(-3)
sample_submission['price'] = blend_predict_sub
sample_submission.to_csv('nn3_submission_blend.csv', index=False)
sample_submission